In [1]:
%config Completer.use_jedi = False

In [2]:
import gym
from gym.wrappers import Monitor
import torch

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt

In [7]:
class cartpole_test(nn.Module):
    def __init__(self, input_sz, hidden_sz,out):
        super().__init__()
        self.input_size=input_sz
        self.hidden_size=hidden_sz
        self.output_size=out
        self.lin1=nn.Linear(self.input_size,self.hidden_size)
        self.lin2=nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, X):
        X=torch.Tensor(X)
        out1 = F.relu(self.lin1(X))
        out2 = F.softmax(self.lin2(out1),dim=-1)
        dist=Categorical(out2)
        action=dist.sample()
        log_probs=dist.log_prob(action)
        return action.item(), log_probs

In [8]:
env=Monitor(gym.make('CartPole-v0'), './video', force=True)

In [9]:
CartPole_test=cartpole_test(env.observation_space.shape[0],16,env.action_space.n)
CartPole_test.load_state_dict(torch.load('model_cart.pt'))
CartPole_test.eval()

cartpole_test(
  (lin1): Linear(in_features=4, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=2, bias=True)
)

In [11]:
max_steps=1000

In [12]:
final_reward=0 
state=env.reset()
rewards=0
for i in range(max_steps):
    action,_= CartPole_test.forward(state)
    next_state, reward, done, _=env.step(action)
    rewards+=reward
    env.render()
    state=next_state
    if done:
        break
    final_reward=rewards
print(final_reward)
env.close()

199.0
